## Niestety nie udało mi się zaimplementować PWB, poprawiłem strukturę klasy i zaimplementowałem dwie metody losowania wag (w tym Xavier). Z PWB udało mi się jedynie uzyskać macierz A

In [43]:
import numpy as np
import pandas as pd
import seaborn as sns
import math
from matplotlib import pyplot as plt
from sklearn import metrics

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

Funkcja aktywacji

In [3]:
def sigmoid(x):
    return 1/(1+np.exp(-x))
#korzystam z biblioteki np, aby x mogl byc wektorem

Pochodna funkcji aktywacji

In [4]:
def sigmoid_der(x):
    return np.exp(-x)/(1+np.exp(-x))**2

Klasa MLP

In [41]:
class MLP:
    def __init__(self, weights, activation_function, activation_function_der, neurons, inputs, outputs):
        self.activation_function = activation_function
        self.activation_function_der = activation_function_der
        self.neurons = neurons
        self.inputs = inputs
        self.outputs = outputs
        if weights is None:
            self.weights_random()
        elif weights == "Xavier": 
            self.weights_xavier()
        else:
            self.weights = weights
    
    def weights_random(self):
        self.weights = [None for i in range (len(self.neurons)+1)]
        
        self.weights[0] = np.random.random((self.inputs + 1, self.neurons[0]))
        self.weights[-1] = np.random.random((self.neurons[-1] + 1, self.outputs))
        
        for i in range(len(self.neurons) - 1):
            self.weights[i+1] = np.random.random((self.neurons[i]+1, self.neurons[i+1]))
    
    def weights_xavier(self):
        xavier = np.sqrt(6)/np.sqrt(self.inputs+self.outputs)
        self.weights = [None for i in range (len(self.neurons)+1)]
        
        self.weights[0] = np.random.random((self.inputs + 1, self.neurons[0]))
        self.weights[-1] = np.random.random((self.neurons[-1] + 1, self.outputs))
        
        for i in range(len(self.neurons) - 1):
            self.weights[i+1] = np.random.random((self.neurons[i]+1, self.neurons[i+1]))
        
        self.weights = [weight * 2 * xavier - xavier for weight in self.weights]
            
    def predict(self, inp):
        A = self.forward(inp)
        return(A[-1])
    
    def forward(self, inp):
        if inp.ndim == 1:
            inp = inp[None]
        #tworzenie macierzy w przypadku wektora
        
        v = inp      
        A = []
        for weight in self.weights:
            v = np.concatenate( [v, np.ones((v.shape[0],1))], axis = 1)
            A.append(v)
            prediction = v @ weight
            v = self.activation_function(prediction)
            A.append(prediction)
        return A
    
    def backward(self, inp, y, eta):
        A = self.forward(inp)
        E = np.zeros(A.shape)
        v = A[-1,]
        v = v[v != 0]
        E[k-1,0:len(v)] = (sigmoid(v) - y) * sigmoid_der(A[k-1,0:len(v)])

        for i in reversed(range (k-1)):
            for j in range (self.weights[i].shape[1]):
                E[i, j] = sigmoid_der(A[i,j]) * sum(self.weights[i+1][i,q] * E[i+1,j]  for q in range(self.weights[i+1].shape[1]))

In [22]:
train_df = pd.read_csv('mio1/regression/square-simple-training.csv', index_col=0)
test_df = pd.read_csv('mio1/regression/square-simple-test.csv', index_col=0)

In [42]:
model1 = MLP(activation_function = sigmoid,activation_function_der = sigmoid_der, neurons = [5,5], inputs = 1, outputs = 1, weights="Xavier")
model1.forward(np.array([1])[None].T)

[array([[1., 1.]]),
 array([[-2.43792681,  1.78248173, -1.31409643,  0.23014844,  1.61023192]]),
 array([[0.08032593, 0.85600304, 0.21180217, 0.55728448, 0.83344358,
         1.        ]]),
 array([[ 2.60894835, -2.8062801 ,  1.81877229, -3.39449793, -0.94113279]]),
 array([[0.93143526, 0.05698575, 0.86041875, 0.03246786, 0.28067158,
         1.        ]]),
 array([[2.75860478]])]